# Biol 359A | Regularization
### Spring 2025, Week 4
Objectives:
- Understand the Bias-Variance Tradeoff: Learn how increasing model complexity impacts bias and variance.
- Gain intuition for Regularization Techniques: Implement Ridge and LASSO to prevent overfitting.
- Implement Model Evaluation and Selection: Employ train-test splits for optimal model selection.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from ipywidgets import interact, interactive, widgets
import seaborn as sns
import textwrap


Today we will start by working with in-silico data. The code below will generate the data.

In [ ]:
! rm -r week4_regularization/
! git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr25/week4_regularization.git
! cp -r week4_regularization/* .
! ls

In [ ]:
df_degrees = pd.read_csv("data/synthetic_data.csv")
print(df_degrees.head())
sns.scatterplot(df_degrees, x="x", y="y");

We want to build a linear regression model that will help us learn about the system that generated these data. Please choose various coefficients for different complexity of the models

### First order

In [ ]:
x_degrees = df_degrees['x'].values
y_degrees = df_degrees['y'].values

# Function to compute model prediction for different polynomial orders
def compute_polynomial(x, coeffs):
    """Compute polynomial of specified order with given coefficients"""
    result = 0
    for i, coeff in enumerate(coeffs):
        result += coeff * (x ** i)
    return result

# Function to compute error (Mean Squared Error)
def compute_mse(y_true, y_pred):
    """Compute Mean Squared Error"""
    return np.mean((y_true - y_pred) ** 2)

# Order 1 model: y = beta_0 + beta_1*x
@interact(
    beta_0=widgets.FloatSlider(min=-30, max=30, step=0.1, value=0, description='β₀:'),
    beta_1=widgets.FloatSlider(min=-15, max=15, step=0.1, value=0, description='β₁:')
)
def order1_model(beta_0, beta_1):
    # Compute the model prediction
    coeffs = [beta_0, beta_1]
    y_pred = compute_polynomial(x_degrees, coeffs)
    
    # Compute error
    mse = compute_mse(y_degrees, y_pred)
    
    # Sort for plotting smooth curve
    sort_idx = np.argsort(x_degrees)
    x_sorted = x_degrees[sort_idx]
    y_pred_sorted = y_pred[sort_idx]
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(x_degrees, y_degrees, color='blue', alpha=0.6, label='Data')
    plt.plot(x_sorted, y_pred_sorted, color='red', linewidth=2, 
             label=f'y = {beta_0:.1f} + {beta_1:.1f}x')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'First-Order Model (MSE: {mse:.2f})')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    print(f"Mean Squared Error: {mse:.4f}")
    #return mse


### Second order

In [ ]:
# Order 2 model: y = beta_0 + beta_1*x + beta_2*x^2
@interact(
    beta_0=widgets.FloatSlider(min=-30, max=30, step=0.1, value=0, description='β₀:'),
    beta_1=widgets.FloatSlider(min=-15, max=15, step=0.1, value=0, description='β₁:'),
    beta_2=widgets.FloatSlider(min=-5, max=5, step=0.05, value=0, description='β₂:')
)
def order2_model(beta_0, beta_1, beta_2):
    # Compute the model prediction
    coeffs = [beta_0, beta_1, beta_2]
    y_pred = compute_polynomial(x_degrees, coeffs)
    
    # Compute error
    mse = compute_mse(y_degrees, y_pred)
    
    # Sort for plotting smooth curve
    sort_idx = np.argsort(x_degrees)
    x_sorted = x_degrees[sort_idx]
    y_pred_sorted = y_pred[sort_idx]
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(x_degrees, y_degrees, color='blue', alpha=0.6, label='Data')
    plt.plot(x_sorted, y_pred_sorted, color='red', linewidth=2, 
             label=f'y = {beta_0:.1f} + {beta_1:.1f}x + {beta_2:.2f}x²')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Second-Order Model (MSE: {mse:.2f})')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    print(f"Mean Squared Error: {mse:.4f}")
    #return mse

### Third order

In [ ]:
# Order 3 model: y = beta_0 + beta_1*x + beta_2*x^2 + beta_3*x^3
@interact(
    beta_0=widgets.FloatSlider(min=-30, max=30, step=0.1, value=0, description='β₀:'),
    beta_1=widgets.FloatSlider(min=-15, max=15, step=0.1, value=0, description='β₁:'),
    beta_2=widgets.FloatSlider(min=-5, max=5, step=0.05, value=0, description='β₂:'),
    beta_3=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0, description='β₃:')
)
def order3_model(beta_0, beta_1, beta_2, beta_3):
    # Compute the model prediction
    coeffs = [beta_0, beta_1, beta_2, beta_3]
    y_pred = compute_polynomial(x_degrees, coeffs)
    
    # Compute error
    mse = compute_mse(y_degrees, y_pred)
    
    # Sort for plotting smooth curve
    sort_idx = np.argsort(x_degrees)
    x_sorted = x_degrees[sort_idx]
    y_pred_sorted = y_pred[sort_idx]
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(x_degrees, y_degrees, color='blue', alpha=0.6, label='Data')
    plt.plot(x_sorted, y_pred_sorted, color='red', linewidth=2, 
             label=f'y = {beta_0:.1f} + {beta_1:.1f}x + {beta_2:.2f}x² + {beta_3:.2f}x³')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f'Third-Order Model (MSE: {mse:.2f})')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    print(f"Mean Squared Error: {mse:.4f}")
    #return mse

## Co-linearity

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 200

# Generate data where x1 and x2 are perfectly correlated (same pathway)
# but neither is related to the disease
x3 = np.random.normal(0, 1, n_samples)  # The true driver

# Generate x1 (and therefore x2 since they're in perfect correlation)
x1 = np.random.normal(0, 1, n_samples)
x2 = x1 + np.random.normal(0, 0.01, n_samples)  # Adding tiny noise to avoid perfect multicollinearity

# Generate disease state based only on x3
y = 5 * x3 + np.random.normal(0, 1, n_samples)  # Adding some noise to the disease state

# Create a dataframe for visualization
df = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'disease_state': y
})

# Visualize the relationships
plt.figure(figsize=(8, 6))

# Relationship between x1 and x2 (same pathway)
plt.subplot(2, 2, 1)
plt.scatter(df['x1'], df['x2'], alpha=0.6)
plt.title('x1 vs x2 (Same Pathway)')
plt.xlabel('x1 Expression')
plt.ylabel('x2 Expression')
plt.grid(True, alpha=0.3)

# Relationship between x1 and disease
plt.subplot(2, 2, 2)
plt.scatter(df['x1'], df['disease_state'], alpha=0.6)
plt.title('x1 vs Disease State')
plt.xlabel('x1 Expression')
plt.ylabel('Disease State')
plt.grid(True, alpha=0.3)

# Relationship between x2 and disease
plt.subplot(2, 2, 3)
plt.scatter(df['x2'], df['disease_state'], alpha=0.6)
plt.title('x2 vs Disease State')
plt.xlabel('x2 Expression')
plt.ylabel('Disease State')
plt.grid(True, alpha=0.3)

# Relationship between x3 and disease
plt.subplot(2, 2, 4)
plt.scatter(df['x3'], df['disease_state'], alpha=0.6)
plt.title('x3 vs Disease State (True Driver)')
plt.xlabel('x3 Expression')
plt.ylabel('Disease State')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Now let's fit the model
X = df[['x1', 'x2', 'x3']].values
y = df['disease_state'].values

model = LinearRegression()
model.fit(X, y)

# Get the coefficients
coefficients = model.coef_

print("Model Coefficients:")
print(f"β1 (x1): {coefficients[0]:.4f}")
print(f"β2 (x2): {coefficients[1]:.4f}")
print(f"β3 (x3): {coefficients[2]:.4f}")
print("\nTrue relationships:")
print("- x1 and x2 are in the same pathway")
print("- x1 and x2 have no effect on disease")
print("- x3 is the true driver with effect = 5")

In [ ]:
# Run multiple simulations to see the distribution of β1 and β2
n_simulations = 500
beta1_values = []
beta2_values = []
beta3_values = []

for i in range(n_simulations):
    # Generate new random data for each simulation
    np.random.seed(i)
    
    # Generate true driver
    x3 = np.random.normal(0, 1, n_samples)
    
    # Generate x1 and x2 (same pathway but unrelated to disease)
    x1 = np.random.normal(0, 1, n_samples)
    x2 = x1 + np.random.normal(0, 0.01, n_samples)
    
    # Generate disease state based only on x3
    y = 5 * x3 + np.random.normal(0, 1, n_samples)
    
    # Fit model
    X = np.column_stack([x1, x2, x3])
    model = LinearRegression()
    model.fit(X, y)
    
    # Store coefficients
    beta1_values.append(model.coef_[0])
    beta2_values.append(model.coef_[1])
    beta3_values.append(model.coef_[2])

# Create a dataframe of coefficient values
coef_df = pd.DataFrame({
    'β1': beta1_values,
    'β2': beta2_values,
    'β3': beta3_values
})

# Plot the distribution of β1 and β2
plt.figure(figsize=(7, 3))

plt.subplot(1, 2, 1)
sns.histplot(coef_df['β1'], kde=True)
plt.axvline(x=0, color='red', linestyle='--')
plt.title('Distribution of β1 Estimates')
plt.xlabel('β1 Value')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
sns.histplot(coef_df['β2'], kde=True)
plt.axvline(x=0, color='red', linestyle='--')
plt.title('Distribution of β2 Estimates')
plt.xlabel('β2 Value')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot the relationship between β1 and β2
plt.figure(figsize=(5, 5))
plt.scatter(coef_df['β1'], coef_df['β2'], alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='red', linestyle='--', alpha=0.5)
plt.title('Relationship Between β1 and β2 Estimates')
plt.xlabel('β1 Value')
plt.ylabel('β2 Value')
plt.grid(True, alpha=0.3)

# Add correlation value
corr = coef_df['β1'].corr(coef_df['β2'])
plt.annotate(f'Correlation: {corr:.4f}', 
             xy=(0.05, 0.95), 
             xycoords='axes fraction',
             bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))

# Calculate the sum of β1 and β2 for each simulation
coef_df['β1 + β2'] = coef_df['β1'] + coef_df['β2']

# Plot the distribution of β1 + β2
plt.figure(figsize=(4, 3))
sns.histplot(coef_df['β1 + β2'], kde=True)
plt.axvline(x=0, color='red', linestyle='--')
plt.title('Distribution of β1 + β2 (Sum of Coefficients)')
plt.xlabel('β1 + β2 Value')
plt.grid(True, alpha=0.3)
plt.show()

# Create a summary of the findings
print("\n----- ANALYSIS OF POSSIBLE VALUES FOR β1 AND β2 -----")
print(f"Mean β1: {coef_df['β1'].mean():.4f}")
print(f"Mean β2: {coef_df['β2'].mean():.4f}")
print(f"Mean β3: {coef_df['β3'].mean():.4f}")
print(f"Standard deviation of β1: {coef_df['β1'].std():.4f}")
print(f"Standard deviation of β2: {coef_df['β2'].std():.4f}")
print(f"Correlation between β1 and β2: {corr:.4f}")
print(f"Mean of β1 + β2: {coef_df['β1 + β2'].mean():.4f}")
print(f"Standard deviation of β1 + β2: {coef_df['β1 + β2'].std():.4f}")

print("\nPossible values of β1 and β2:")
print("1. The true values of both β1 and β2 are 0 (since they don't affect the disease).")
print("2. However, due to collinearity, the estimates can take many values.")
print("3. The estimates of β1 and β2 are highly negatively correlated.")
print("4. β1 and β2 act like a seesaw: when one goes up, the other goes down.")
print("5. The sum β1 + β2 tends to be close to zero (the true combined effect).")
print("6. Any combination where β1 ≈ -β2 is a possible outcome.")
print("7. The model cannot distinguish between the effects of x1 and x2.")
print("8. In some simulations, both β1 and β2 can appear statistically significant despite having no true effect.")